# Labcodeset
Load used libraries


In [1]:
#!pip install xlsxwriter
import getpass
import requests
import pandas as pd
from IPython.display import JSON

Get credentials for the National Terminology Server

In [2]:
# username=input("NTS username: ")
username = 'marcdegraauw@gmail.com'
password = getpass.getpass("NTS password: ")

NTS password:  ··········


Get an access token using the credentials

In [15]:
url = "https://terminologieserver.nl/auth/realms/nictiz/protocol/openid-connect/token"

payload='username='+username+'&password='+password+'&grant_type=password&client_id=cli_client'
headers = { 'Content-Type': 'application/x-www-form-urlencoded' }

response = requests.request("POST", url, headers=headers, data=payload)

token = response.json()
access_token = token["access_token"]

#print(access_token)

In [16]:
def lookupcode(code):
    url = "https://terminologieserver.nl/" + "fhir/CodeSystem/$lookup?system=http://labterminologie.nl/cs/labconcepts&property=*&code=" + code
    headers = { 'Authorization': 'Bearer ' + access_token }
    response = requests.request("GET", url, headers=headers, data=payload)
    return response.json()
# lookupcode('LP7370-2')
lookupcode('17-4')

{'resourceType': 'Parameters',
 'parameter': [{'name': 'name', 'valueString': 'Labconcepts'},
  {'name': 'system',
   'valueString': 'http://labterminologie.nl/cs/labconcepts'},
  {'name': 'version', 'valueString': '20210503'},
  {'name': 'display',
   'valueString': 'Amoxicilline [gevoeligheid] in isolaat d.m.v. agardiffusie'},
  {'name': 'designation',
   'part': [{'name': 'language', 'valueCode': 'en'},
    {'name': 'use',
     'valueCoding': {'system': 'http://snomed.info/sct',
      'code': '900000000000013009',
      'display': 'Synonym'}},
    {'name': 'value',
     'valueString': 'Amoxicillin [Susceptibility] by Disk diffusion (KB)'}]},
  {'name': 'designation',
   'part': [{'name': 'language', 'valueCode': 'nl-NL'},
    {'name': 'use',
     'valueCoding': {'system': 'http://terminology.hl7.org/CodeSystem/designation-usage',
      'code': 'display'}},
    {'name': 'value',
     'valueString': 'Amoxicilline [gevoeligheid] in isolaat d.m.v. agardiffusie'}]},
  {'name': 'property'

In [17]:
def displaynl(code):
    data = lookupcode(code)
    try:
        for param in data['parameter']:
            if param['name'] == 'display':
                return param['valueString']
    except KeyError:
        pass
    return None
displaynl('LP65332-6')

'allantoïne'

In [18]:
def lookupmaterial(code):
    url = "https://terminologieserver.nl/" + "fhir/ConceptMap/$translate?url=http://labterminologie.nl/cm/labconcepts-materials&system=http://loinc.org&target=http://snomed.info/sct&code=" + code
    headers = { 'Authorization': 'Bearer ' + access_token }
    response = requests.request("GET", url, headers=headers, data=payload)
    return response.json()

def displaymat(code):
    data = lookupmaterial(code)
    try:
        return data['parameter'][1]['part'][1]['valueCoding']['display']
    except KeyError:
        pass
    return None
displaymat('73837-7')

'serum'

Search ValueSets with title TNMVersieCodeLijst

In [21]:
loinc = "17-4"
data = lookupcode(loinc)
JSON(data)

<IPython.core.display.JSON object>

In [22]:
for param in data['parameter']:
    if not('name' in param):
        print('Error')
    elif param['name'] == 'designation':
        lang = param['part'][0]['valueCode']
        print(param['part'][2]['valueString'] + ' (' + lang + ')')
    elif param['name'] == 'property':
        for key in param.keys():
            if key == 'part':
                for pt in param[key][0]:
                    if 'name' in pt:
                        pass
                    else:
                        name = param[key][0][pt]
                for pt in param[key][1]:
                    if 'name' in pt:
                        pass
                    else:
                        value = param[key][1][pt]
                        if 'code' in value:
                            nl = displaynl(value['code'])
                            if nl is not None:
                                value = nl
                            if name == 'EXAMPLE_UCUM_UNITS':
                                value = value['code']
                            if name == 'MATERIAL':
                                nlmat = displaymat(loinc)
                                if nlmat is not None:
                                    value = nlmat
                print(name + ': ', value)
    else:
        for pt in param:
            if pt == 'name':
                name = param[pt]
            else:
                value = param[pt]
        print(name + ': ', value)

name:  Labconcepts
system:  http://labterminologie.nl/cs/labconcepts
version:  20210503
display:  Amoxicilline [gevoeligheid] in isolaat d.m.v. agardiffusie
Amoxicillin [Susceptibility] by Disk diffusion (KB) (en)
Amoxicilline [gevoeligheid] in isolaat d.m.v. agardiffusie (nl-NL)
SYSTEM:  isolaat
TIME_ASPCT:  moment
PROPERTY:  gevoeligheid
METHOD_TYP:  agardiffusie
SCALE_TYP:  ordinaal-kwantitatief
CLASS:  ABXBACT
STATUS:  active
EXAMPLE_UCUM_UNITS:  mm
COMPONENT:  amoxicilline
MATERIAL:  isolaat van micro-organisme
